In [11]:
import os
import re
import sys

current_dir = os.getcwd()
sys.path.append(os.path.abspath(os.path.join(current_dir, '..', 'PVT-thermal-performance-model')))
sys.path.append(os.path.abspath(os.path.join(current_dir, '..', 'PVT-thermal-performance-model', 'CFD')))
sys.path.append(os.path.abspath(os.path.join(current_dir, '..', 'RD-systems-and-test-benches')))
sys.path.append(os.path.abspath(os.path.join(current_dir, '..', 'PVT-PL-model')))
sys.path.append(os.path.abspath(os.path.join(current_dir, '..', 'PVT-PL-model', 'ansys')))

import numpy as np
import pandas as pd
import openpyxl

from scipy import interpolate
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import plotly.graph_objects as go
import plotly.io as pio
# import plotly 
# plotly.io.kaleido.scope.mathjax= None

from tqdm import tqdm

from CoolProp.CoolProp import PropsSI

# Own modules

import jou_gen as jg
import hx_hydraulic as hxhy
import model_fsolve_v2 as modf2

import utils.conversion as conversion

In [12]:
import repo_config as rconfig

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
from matplotlib.ticker import FuncFormatter
import matplotlib.gridspec as gridspec
import matplotlib_inline

import scienceplots

matplotlib_inline.backend_inline.set_matplotlib_formats('retina')
plt.style.use(['science'])

def to_percent(y, _):
    return f"{y * 100:.0f}\%"

# colors = ['olive', 'green', 'blue', 'orange', 'red', 'grey','yellow','bordeaux']
colors = ['olive','orange','blue','red','green','grey']
colors70 = [rconfig.get_hex_code(color, 70) for color in colors]
colors = [rconfig.get_hex_code(color, 100) for color in colors]
linestyles = ['-', '--',  (5, (10, 3)), '-.', (0, (1, 1)), (0, (3, 5, 1, 5)), (0, (3, 5, 1, 5)), (0, (3, 5, 1, 5))]
markers = ['o', 's', 'd', 'X', 'v', '^', 'P', 'H']

In [13]:
def sort_key(path):
    match = re.search(r'case(\d+)', path)
    if match:
        return (path.split('case')[0], int(match.group(1)))
    return (path, 0)

In [14]:
# abaque_folder = r'G:\Drive partagés\Cercle Hard\R&D\Modèles numériques PVT\ANSYS Fluent Projects\Hydraulics\NUMAIRO_V4.5_motif\SimulationResults\abaque0_laminar_try0'
abaque_folder = r'G:\Drive partagés\Cercle Hard\R&D\Modèles numériques PVT\ANSYS Fluent Projects\Hydraulics\NUMAIRO_motif\SimulationResults\V4.5_1MPE_M2\abaque300_laminar_try0'

cases_paths = [f.path for f in os.scandir(abaque_folder) if f.is_dir()]
cases_paths.sort(key=sort_key)
# cases_paths

In [15]:
Vdot = 300

In [16]:
T_op = 23 + 273.15
p_op = 101325

rho_op = PropsSI('D', 'T', T_op, 'P', p_op, 'INCOMP::MPG[0.4]')
rho_op

1030.5622161622794

In [17]:
list_dict = []

for i, case_path in tqdm(enumerate(cases_paths), total=len(cases_paths)):

    sp_df = pd.read_csv(os.path.join(cases_paths[i], 'sp_report.csv'), sep=',')
    mdot_df = pd.read_csv(os.path.join(cases_paths[i], 'mdot_report.csv'), sep=',')
    
    list_dict.append(
                {
                'case_name' : f'case{i}',
                'mdot_max' : mdot_df.loc[mdot_df['Component'] == 'distrib_inlet', 'mdot'].values[0] + mdot_df.loc[mdot_df['Component'] == 'coll_inlet', 'mdot'].values[0],
                'Vdot_max' : ((mdot_df.loc[mdot_df['Component'] == 'distrib_inlet', 'mdot'].values[0] + mdot_df.loc[mdot_df['Component'] == 'coll_inlet', 'mdot'].values[0])/rho_op) * 3.6e6,
                'mdot_distrib_inlet' : mdot_df.loc[mdot_df['Component'] == 'distrib_inlet', 'mdot'].values[0],
                'mdot_distrib_outlet' : - mdot_df.loc[mdot_df['Component'] == 'distrib_outlet', 'mdot'].values[0],
                'mdot_coll_inlet' : mdot_df.loc[mdot_df['Component'] == 'coll_inlet', 'mdot'].values[0],
                'mdot_coll_outlet' : - mdot_df.loc[mdot_df['Component'] == 'coll_outlet', 'mdot'].values[0],
                'alpha' : 0.,
                'beta' : mdot_df.loc[mdot_df['Component'] == 'distrib_inlet', 'mdot'].values[0] / (mdot_df.loc[mdot_df['Component'] == 'distrib_inlet', 'mdot'].values[0] + mdot_df.loc[mdot_df['Component'] == 'coll_inlet', 'mdot'].values[0]),
                'P_coll_inlet' : sp_df.loc[sp_df['Component'] == 'coll_inlet', 'sp'].values[0],
                'P_coll_outlet' : sp_df.loc[sp_df['Component'] == 'coll_outlet', 'sp'].values[0],
                'P_distrib_inlet' : sp_df.loc[sp_df['Component'] == 'distrib_inlet', 'sp'].values[0],
                'P_distrib_outlet' : sp_df.loc[sp_df['Component'] == 'distrib_outlet', 'sp'].values[0]
                  }
                  )
    
abaque_df = pd.DataFrame(list_dict)
abaque_df['alpha'] = 1 - (abaque_df['mdot_distrib_outlet'] / abaque_df['mdot_distrib_inlet'])
abaque_df['DPd'] = abaque_df['P_distrib_inlet'] - abaque_df['P_distrib_outlet']
abaque_df['DPc'] = abaque_df['P_coll_inlet'] - abaque_df['P_coll_outlet']
abaque_df['DPdc'] = abaque_df['P_distrib_inlet'] - abaque_df['P_coll_inlet']
abaque_df['Qin_d'] = abaque_df['mdot_distrib_inlet'] / rho_op # m3/s kg/s / kg/m3
abaque_df

100%|██████████| 25/25 [00:31<00:00,  1.28s/it]


,case_name,mdot_max,Vdot_max,mdot_distrib_inlet,mdot_distrib_outlet,mdot_coll_inlet,mdot_coll_outlet,alpha,beta,Pin_c,Pout_c,Pin_d,Pout_d,DPd,DPc,DPdc,Qin_d
0,case0,0.08588,299.999999,0.008588,0.008159,0.077292,0.077721,0.049962,0.1,37.782909,0.000713,164.05856,162.58024,1.47832,37.782196,126.275651,0.000008
1,case1,0.08588,299.999999,0.008588,0.006442,0.077292,0.079438,0.249916,0.1,43.649771,0.001041,779.19143,778.14771,1.04372,43.648730,735.541659,0.000008
2,case2,0.08588,299.999999,0.008588,0.004296,0.077292,0.081585,0.499816,0.1,51.788834,0.001562,1549.55450,1548.91580,0.63870,51.787272,1497.765666,0.000008
3,case3,0.08588,299.999999,0.008588,0.002152,0.077292,0.083728,0.749394,0.1,59.218892,0.000961,2320.65190,2320.29150,0.36040,59.217931,2261.433008,0.000008
4,case4,0.08588,299.999999,0.008588,0.000441,0.077292,0.085439,0.948644,0.1,65.059179,0.001186,2938.61110,2938.37970,0.23140,65.057993,2873.551921,0.000008
5,case5,0.08588,299.999998,0.025764,0.024478,0.060116,0.061403,0.049934,0.3,28.322809,0.000596,471.79177,465.30992,6.48185,28.322213,443.468961,0.000025
6,case6,0.08588,299.999998,0.025764,0.019327,0.060116,0.066553,0.249860,0.3,42.886833,0.000573,2318.79190,2315.13770,3.65420,42.886260,2275.905067,0.000025
7,case7,0.08588,299.999998,0.025764,0.012893,0.060116,0.072987,0.499580,0.3,60.646444,0.000215,4653.02770,4651.96620,1.06150,60.646229,4592.381256,0.000025
8,case8,0.08588,299.999998,0.025764,0.006488,0.060116,0.079392,0.748178,0.3,79.457054,0.000880,7022.61550,7023.20110,-0.58560,79.456174,6943.158446,0.000025
9,case9,0.08588,299.999998,0.025764,0.001558,0.060116,0.084322,0.939519,0.3,95.582716,0.000589,8881.36720,8882.70760,-1.34040,95.582127,8785.784484,0.000025


### Test sur 1 cas STRESSS

In [18]:
def create_interpolating_function(df, X1_name, X2_name, Y_name, degree, include_bias):

    # Generate polynomial features up to the third degree
    X_columns = df[[X1_name, X2_name]]
    poly = PolynomialFeatures(degree=degree, include_bias=include_bias)
    X_poly = poly.fit_transform(X_columns)
    # Fit the model
    model = LinearRegression()
    model.fit(X_poly, df[Y_name])
    # Coefficients
    coefficients = model.coef_
    intercept = model.intercept_

    def interpolate_Y(X1_value, X2_value):
        # Combine the input values into the same structure as the model expects
        X_input = np.array([[X1_value, X2_value]])
        # Transform the input values to polynomial features
        X_input_poly = poly.transform(X_input)
        # Predict the output using the fitted model
        Y_pred = model.predict(X_input_poly)
        return Y_pred[0]
    
    return interpolate_Y

In [19]:
# df_abaque = pd.read_excel(r"G:\Drive partagés\Cercle Hard\Notion\PRJ-1088 - CFD pour la caractérisation hydraulique d’un échangeur à mini-canaux\CFD_V4.5_1MPE_woch\AbaqueCFD_1MPE_woch.xlsx")

N=1

df_fil = abaque_df
X1_name = 'alpha'
X2_name = 'beta'
X_columns = df_fil[[X1_name, X2_name]]
poly = PolynomialFeatures(degree=3, include_bias=False)
X_poly = poly.fit_transform(X_columns)

# model_d = LinearRegression()
# model_d.fit(X_poly, df_fil['DPd'])
# model_c = LinearRegression()
# model_c.fit(X_poly, df_fil['DPc'])
# model_dc = LinearRegression()
# model_dc.fit(X_poly, df_fil['DPdc'])

# def DPd(Qin, alpha):
#     point = np.array([[Qin*3600000/Vdot, alpha]])
#     X_poly = poly.fit_transform(point)
#     return model_d.predict(X_poly)[0]
# def DPc(Qin, alpha):
#     point = np.array([[Qin*3600000/Vdot, alpha]])
#     X_poly = poly.fit_transform(point)
#     return model_c.predict(X_poly)[0]
# def DPdc(Qin, alpha):
#     point = np.array([[Qin*3600000/Vdot, alpha]])
#     X_poly = poly.fit_transform(point)
#     return - model_dc.predict(X_poly)[0]

DPd, DPc, DPdc = modf2.transfer_func(df_fil, deg_a = 2, deg_q = 2)

def DP_beta(letter, alpha, beta):
    # Define a dictionary to map letters to functions
    dp_functions = {
        'd': DPd,
        'c': DPc,
        'dc': DPdc
    }
    
    # Call the appropriate function if the letter is valid
    if letter in dp_functions:
        return dp_functions[letter](beta * Q_tot, alpha)
    else:
        raise ValueError(f"Invalid letter '{letter}'. Expected one of: {', '.join(dp_functions.keys())}")


# DPd = create_interpolating_function(df_fil, 'Qin_d', 'alpha', 'DPd', 3, False)
# DPc = create_interpolating_function(df_fil, 'Qin_d', 'alpha', 'DPc', 3, False)
# DPdc = create_interpolating_function(df_fil, 'Qin_d', 'alpha', 'DPdc', 3, False)

tabl, residuals = modf2.PL_fsolve_MPE(N*19, Vdot/3600000, DPd, DPc, DPdc)

print(tabl)

            Pin        Pout         qx
0   2115.207238  454.485791  15.664906
1   2080.446692  452.252940  15.405097
2   2048.591197  448.013232  15.192660
3   2019.499475  441.691879  15.027148
4   1993.040981  433.204375  14.908305
5   1969.094507  422.455242  14.836053
6   1947.546810  409.336640  14.810496
7   1928.291263  393.726807  14.831911
8   1911.226493  375.488285  14.900747
9   1896.254980  354.465869  15.017611
10  1883.281604  330.484189  15.183257
11  1872.212091  303.344770  15.398552
12  1862.951316  272.822291  15.664423
13  1855.401387  238.659519  15.981751
14  1849.459377  200.559708  16.351168
15  1845.014405  158.173490  16.772665
16  1841.943286  111.071440  17.245003
17  1840.102060   58.670728  17.767328
18  1839.300338   -0.167893  18.414497


g:\Mon Drive\GitHub\PVT-PL-model\model_fsolve_v2.py:431: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  Xsol = sc.fsolve(FUN, X0)
